In [1]:
# Jupyter Notebook: شبیه‌سازی دیتاست مولتی‌مودال نزدیک به واقعیت
# شامل: ویدیو، کیبورد/ماوس، متن و برچسب‌های تمرکز / حواس‌پرت / خسته

import numpy as np
import tensorflow as tf

# ----------------------- تنظیمات دیتاست -----------------------
NUM_SAMPLES = 2000  # تعداد نمونه‌ها
VIDEO_FRAMES = 8
VIDEO_H = 112
VIDEO_W = 112
VIDEO_C = 3
SENSOR_SEQ = 128
SENSOR_FEATS = 4
TEXT_SEQ = 64
VOCAB_SIZE = 20000
NUM_CLASSES = 3

# ----------------------- توزیع برچسب‌ها -----------------------
# 0: Focus, 1: Distracted, 2: Fatigued
labels = np.random.choice(NUM_CLASSES, NUM_SAMPLES, p=[0.5, 0.3, 0.2])  # نزدیک به واقعیت

# ----------------------- شبیه‌سازی ویدیو -----------------------
# هر ویدیو: frames x H x W x C با نویز تصادفی و الگوهای ساده
X_video = np.zeros((NUM_SAMPLES, VIDEO_FRAMES, VIDEO_H, VIDEO_W, VIDEO_C), dtype='float32')
for i in range(NUM_SAMPLES):
    base = np.random.rand(VIDEO_H, VIDEO_W, VIDEO_C) * 0.5  # زمینه اولیه
    for f in range(VIDEO_FRAMES):
        motion = np.random.rand(VIDEO_H, VIDEO_W, VIDEO_C) * 0.5
        X_video[i, f] = np.clip(base + motion * (labels[i]+1)/3.0, 0, 1)  # تغییر براساس کلاس

# ----------------------- شبیه‌سازی کیبورد/ماوس -----------------------
# 4 ویژگی: [کلید/کلیک سرعت, وقفه بین کلیدها, سرعت موس, jitter موس]
X_sensor = np.zeros((NUM_SAMPLES, SENSOR_SEQ, SENSOR_FEATS), dtype='float32')
for i in range(NUM_SAMPLES):
    if labels[i] == 0:  # Focus
        X_sensor[i] = np.random.normal(loc=[0.8,0.2,0.8,0.1], scale=0.05, size=(SENSOR_SEQ,SENSOR_FEATS))
    elif labels[i] == 1:  # Distracted
        X_sensor[i] = np.random.normal(loc=[0.5,0.5,0.5,0.3], scale=0.07, size=(SENSOR_SEQ,SENSOR_FEATS))
    else:  # Fatigued
        X_sensor[i] = np.random.normal(loc=[0.3,0.7,0.3,0.4], scale=0.05, size=(SENSOR_SEQ,SENSOR_FEATS))

# ----------------------- شبیه‌سازی متن -----------------------
# متن به صورت توکن‌های تصادفی با طول TEXT_SEQ
X_text = np.zeros((NUM_SAMPLES, TEXT_SEQ), dtype='int32')
for i in range(NUM_SAMPLES):
    if labels[i] == 0:  # Focus
        X_text[i] = np.random.randint(0, VOCAB_SIZE//2, size=TEXT_SEQ)
    elif labels[i] == 1:  # Distracted
        X_text[i] = np.random.randint(0, VOCAB_SIZE, size=TEXT_SEQ)
    else:  # Fatigued
        X_text[i] = np.random.randint(VOCAB_SIZE//2, VOCAB_SIZE, size=TEXT_SEQ)

# ----------------------- one-hot برچسب‌ها -----------------------
y = tf.keras.utils.to_categorical(labels, NUM_CLASSES)

# ----------------------- ذخیره دیتاست -----------------------
np.savez('synthetic_multimodal_dataset.npz', X_video=X_video, X_sensor=X_sensor, X_text=X_text, y=y)

print("دیتاست مولتی‌مودال شبیه‌سازی شد و در 'synthetic_multimodal_dataset.npz' ذخیره شد")
print("نمونه shapes:")
print("X_video:", X_video.shape)
print("X_sensor:", X_sensor.shape)
print("X_text:", X_text.shape)
print("y:", y.shape)


دیتاست مولتی‌مودال شبیه‌سازی شد و در 'synthetic_multimodal_dataset.npz' ذخیره شد
نمونه shapes:
X_video: (2000, 8, 112, 112, 3)
X_sensor: (2000, 128, 4)
X_text: (2000, 64)
y: (2000, 3)
